In [115]:
# Import thu vien
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [116]:
# doc file csv Customer_Transaction
df = pd.read_csv("raw_data\Customer_Transaction.csv")

In [117]:
df.head()

,ID,CustomerID,Purchase_Date,GMV
0,0,1327813,2022-06-01 00:00:00.000,95000
1,1,1157830,2022-06-01 00:00:00.000,75000
2,2,873915,2022-07-01 00:00:00.000,95000
3,3,3505071,2022-07-01 00:00:00.000,90000
4,4,2930918,2022-07-01 00:00:00.000,109091


In [118]:
# tong cac gia tri khong NaN trong tung cot
df.count()

ID               1048575
CustomerID       1048575
Purchase_Date    1048575
GMV              1048575
dtype: int64

In [119]:
# loai du lieu cac cot
df.dtypes

ID                int64
CustomerID        int64
Purchase_Date    object
GMV               int64
dtype: object

In [120]:
# Chuyen type cua Purchase_Date sang date
df['Purchase_Date'] =  pd.to_datetime(df['Purchase_Date'], format='%Y-%m-%d %H:%M:%S.%f')

In [121]:
df.dtypes

ID                        int64
CustomerID                int64
Purchase_Date    datetime64[ns]
GMV                       int64
dtype: object

In [122]:
df.tail()

,ID,CustomerID,Purchase_Date,GMV
1048570,1048570,4127818,2022-06-01,70000
1048571,1048571,1987234,2022-07-01,75000
1048572,1048572,3283899,2022-07-01,70000
1048573,1048573,3985289,2022-07-01,70000
1048574,1048574,3189801,2022-07-28,0


In [123]:
df.describe()

,ID,CustomerID,Purchase_Date,GMV
count,1.048575e+06,1.048575e+06,1048575,1.048575e+06
mean,5.242870e+05,2.615855e+06,2022-07-02 05:01:30.297403904,7.892864e+04
min,0.000000e+00,0.000000e+00,2022-06-01 00:00:00,0.000000e+00
25%,2.621435e+05,1.713096e+06,2022-06-01 00:00:00,7.000000e+04
50%,5.242870e+05,2.780590e+06,2022-07-01 00:00:00,7.500000e+04
75%,7.864305e+05,3.604265e+06,2022-08-01 00:00:00,9.500000e+04
max,1.048574e+06,4.392919e+06,2022-08-31 00:00:00,8.400000e+06
std,3.026977e+05,1.142409e+06,NaN,2.365348e+04


In [124]:
# bo CustomerID = 0
df = df[df['CustomerID'] != 0][df['GMV'] > 0]

C:\Users\letha\AppData\Local\Temp\ipykernel_16808\3498133713.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['CustomerID'] != 0][df['GMV'] > 0]


In [125]:
df.describe()

,ID,CustomerID,Purchase_Date,GMV
count,1.040158e+06,1.040158e+06,1040158,1.040158e+06
mean,5.242571e+05,2.610880e+06,2022-07-02 04:43:41.370984704,7.956695e+04
min,0.000000e+00,7.173900e+04,2022-06-01 00:00:00,2.420000e+02
25%,2.621022e+05,1.707394e+06,2022-06-01 00:00:00,7.000000e+04
50%,5.242525e+05,2.770588e+06,2022-07-01 00:00:00,7.500000e+04
75%,7.864028e+05,3.596549e+06,2022-08-01 00:00:00,9.500000e+04
max,1.048573e+06,4.392919e+06,2022-08-31 00:00:00,8.400000e+06
std,3.026993e+05,1.142413e+06,NaN,2.265503e+04


In [126]:
# doc file csv Customer_Registered
#df_r = pd.read_csv("raw_data\Customer_Registered.csv")

In [127]:
#df_r.head()

In [128]:
#df_r.describe()

In [129]:
# Chuyen type cua created_date sang date
#df_r['created_date'] =  pd.to_datetime(df_r['created_date'], format='%Y-%m-%d %H:%M:%S.%f')

In [130]:
#df_r.dtypes

In [131]:
# Inner join on CustomerID
#joined_df = pd.merge(left=df, right=df_r, left_on='CustomerID', right_on='ID', how='inner')

In [132]:
#joined_df.head()

In [133]:
# lay ngay lon nhat trong Purchase_Date + 1 -> de tinh Recency
current_date = max(df['Purchase_Date']) + datetime.timedelta(days=1)

# Group by CustomerID de tinh R, F, M
df_customer = df.groupby(['CustomerID']).agg(
    {'Purchase_Date': lambda x: (current_date- x.max()).days, # ngay mua gan nhat
     'ID':'count', # so lan mua
     'GMV':'sum' # tong gia tri mua hang
    }
)


In [134]:
df_customer.head()

,Purchase_Date,ID,GMV
CustomerID,,,
71739,62,1,105000
72014,62,2,254091
72052,31,1,145000
72657,62,1,200000
74549,62,1,125000


In [135]:
# doi ten cot
df_customer.rename(columns={'Purchase_Date':'Recency','ID':'Frequency','GMV':'Monetary'}, inplace=True)

In [136]:
df_customer.describe()

,Recency,Frequency,Monetary
count,936660.000000,936660.000000,9.366600e+05
mean,60.423782,1.110497,8.835885e+04
std,26.234584,0.323117,3.662808e+04
min,1.000000,1.000000,2.420000e+02
25%,31.000000,1.000000,7.000000e+04
50%,62.000000,1.000000,7.500000e+04
75%,92.000000,1.000000,9.500000e+04
max,92.000000,5.000000,8.400000e+06


In [137]:
# Ve phan phoi de kiem tra
'''
fig, ax = plt.subplots(1, 3, figsize=(12,4))
sns.distplot(df_customer['Recency'], ax = ax[0])
sns.distplot(df_customer['Frequency'], ax = ax[1])
sns.distplot(df_customer['Monetary'], ax = ax[2])
plt.show()
'''

"\nfig, ax = plt.subplots(1, 3, figsize=(12,4))\nsns.distplot(df_customer['Recency'], ax = ax[0])\nsns.distplot(df_customer['Frequency'], ax = ax[1])\nsns.distplot(df_customer['Monetary'], ax = ax[2])\nplt.show()\n"

In [138]:
from scipy import stats

In [139]:
# kiem tra do lech cua phan phoi
def analyze_skewness(x):
    print(df_customer[x].skew().round(2))
    print(np.log(df_customer[x]).skew().round(2))
    print(np.sqrt(df_customer[x]).skew().round(2))
    print(pd.Series(stats.boxcox(df_customer[x])[0]).skew().round(2))

In [140]:
analyze_skewness('Recency')

0.02
-0.87
-0.19
-0.13


In [141]:
analyze_skewness('Frequency')

2.78
2.6
2.66
2.53


In [142]:
analyze_skewness('Monetary')

25.15
-2.01
1.2
0.32
